In [13]:
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [14]:
macro = pd.read_csv("../data/processed/macro_processed.csv", parse_dates=["Unnamed: 0"])
macro = macro.rename(columns={"Unnamed: 0": "Date"})
macro = macro.set_index("Date")
macro.head()



,DGS10,INDPRO,UNRATE,CPIAUCSL,DCOILWTICO,FEDFUNDS,VIXCLS,UMCSENT,GDP
Date,,,,,,,,,
2000-01-03,6.58,91.8239,4.1,170.0,25.56,5.73,24.21,111.3,10660.465
2000-01-04,6.49,91.8239,4.1,170.0,25.56,5.73,27.01,111.3,10660.465
2000-01-05,6.62,91.8239,4.1,170.0,24.65,5.73,26.41,111.3,10660.465
2000-01-06,6.57,91.8239,4.1,170.0,24.79,5.73,25.73,111.3,10660.465
2000-01-07,6.52,91.8239,4.1,170.0,24.79,5.73,21.72,111.3,10660.465


**Goal: Event Response Features**
- Capture systematic market responses to macro catalysts and convert them into reusable features for signal research.

**Personal Notes**
- Unlike traditional event studies which is one-off analysis, average response around an event that are mostly descriptive and not reuseable and hard to integrate into a larger framework, I will be focusing on converting macro events into time indexed features not only on event dates and can feed into predictive models, regime analysis and cross-asset signal research. In simple english, We are not trying to study events but studying how markets encode macro information over time.

**Layer 1**
- Capture slow-moving macro conditions that define the environment in which shocks propagate.

**Why this matters**
- Markets respond differently to the same event depending on the macro state.

In [ ]:
state = pd.DataFrame(index=macro.index)

state["inflation_yoy"] = macro["CPIAUCSL"].pct_change(12) 
state["growth_yoy"] = macro["INDPRO"].pct_change(12)
state["unemployment"] = macro["UNRATE"]
state["sentiment"] = macro["UMCSENT"]

Date
2000-01-03        NaN
2000-01-04        NaN
2000-01-05        NaN
2000-01-06        NaN
2000-01-07        NaN
               ...   
2000-05-15    0.00117
2000-05-16    0.00117
2000-05-17    0.00000
2000-05-18    0.00000
2000-05-19    0.00000
Name: inflation_yoy, Length: 100, dtype: float64

In [10]:
macro.isnull().sum()/len(macro)

DGS10         0.0
INDPRO        0.0
UNRATE        0.0
CPIAUCSL      0.0
DCOILWTICO    0.0
FEDFUNDS      0.0
VIXCLS        0.0
UMCSENT       0.0
GDP           0.0
dtype: float64

In [11]:
macro["month-year"] = macro.index.to_period("M")
macro["qtr-year"] = macro.index.to_period("Q")
macro["bi-annual-year"] = macro.index.to_period("2Q")
macro["year"] = macro.index.to_period("Y")

In [12]:
macro.head()

,DGS10,INDPRO,UNRATE,CPIAUCSL,DCOILWTICO,FEDFUNDS,VIXCLS,UMCSENT,GDP,month-year,qtr-year,bi-annual-year,year
Date,,,,,,,,,,,,,
2000-01-03,6.58,91.8239,4.1,170.0,25.56,5.73,24.21,111.3,10660.465,2000-01,2000Q1,2000Q1,2000
2000-01-04,6.49,91.8239,4.1,170.0,25.56,5.73,27.01,111.3,10660.465,2000-01,2000Q1,2000Q1,2000
2000-01-05,6.62,91.8239,4.1,170.0,24.65,5.73,26.41,111.3,10660.465,2000-01,2000Q1,2000Q1,2000
2000-01-06,6.57,91.8239,4.1,170.0,24.79,5.73,25.73,111.3,10660.465,2000-01,2000Q1,2000Q1,2000
2000-01-07,6.52,91.8239,4.1,170.0,24.79,5.73,21.72,111.3,10660.465,2000-01,2000Q1,2000Q1,2000


In [33]:
df_monthly = df.groupby(["month-year"]).mean()
df_qtrly = df.groupby(["qtr-year"]).mean()
df_biannually = df.groupby(["bi-annual-year"]).mean()
df_yearly = df.groupby(["year"]).mean()

In [34]:
df_monthly.nunique()/len(df_monthly)

GDP               0.227564
UNRATE            0.195513
CPIAUCSL          0.685897
FEDFUNDS          0.394231
DGS10             0.993590
DCOILWTICO        1.000000
INDPRO            0.711538
UMCSENT           0.525641
VIXCLS            0.993590
qtr-year          0.333333
bi-annual-year    0.333333
year              0.083333
dtype: float64

In [35]:
df_qtrly.nunique()/len(df_qtrly)

GDP               0.509615
UNRATE            0.942308
CPIAUCSL          1.000000
FEDFUNDS          0.903846
DGS10             1.000000
DCOILWTICO        1.000000
INDPRO            1.000000
UMCSENT           1.000000
VIXCLS            1.000000
month-year        1.000000
bi-annual-year    1.000000
year              0.250000
dtype: float64

In [36]:
df_biannually.nunique()/len(df_biannually)

GDP           0.509615
UNRATE        0.942308
CPIAUCSL      1.000000
FEDFUNDS      0.903846
DGS10         1.000000
DCOILWTICO    1.000000
INDPRO        1.000000
UMCSENT       1.000000
VIXCLS        1.000000
month-year    1.000000
qtr-year      1.000000
year          0.250000
dtype: float64

In [37]:
df_yearly.nunique()/len(df_yearly)

GDP               0.961538
UNRATE            1.000000
CPIAUCSL          1.000000
FEDFUNDS          1.000000
DGS10             1.000000
DCOILWTICO        1.000000
INDPRO            1.000000
UMCSENT           1.000000
VIXCLS            1.000000
month-year        1.000000
qtr-year          1.000000
bi-annual-year    1.000000
dtype: float64

In [38]:
df_yearly

,GDP,UNRATE,CPIAUCSL,FEDFUNDS,DGS10,DCOILWTICO,INDPRO,UMCSENT,VIXCLS,month-year,qtr-year,bi-annual-year
year,,,,,,,,,,,,
2000,10660.465000,3.992063,172.199603,6.250595,6.029444,30.367579,92.538752,107.283730,23.315000,2000-06,2000Q2,2000Q2
2001,10660.465000,4.675403,176.912500,4.043347,5.019274,25.887097,89.901336,90.173387,25.749677,2001-06,2001Q2,2001Q2
2002,10902.125444,5.758730,179.814286,1.705873,4.606944,26.190198,89.968779,89.041270,27.292460,2002-06,2002Q2,2002Q2
2003,11433.101044,6.000397,183.482937,1.153095,4.016865,31.079881,91.089392,88.734921,21.982857,2003-06,2003Q2,2003Q2
2004,12183.152310,5.557540,188.710714,1.336706,4.270754,41.507976,93.482965,94.569841,15.480397,2004-06,2004Q2,2004Q2
2005,12937.799889,5.095238,195.288492,3.181429,4.290159,56.644048,96.616181,88.915476,12.807063,2005-06,2005Q2,2005Q2
2006,13142.642000,4.623904,201.399203,4.923705,4.794622,66.051116,98.979297,86.685259,12.806534,2006-06,2006Q2,2006Q2
2007,13543.576757,4.550996,207.149028,5.044104,4.632430,72.386693,101.543151,84.821912,17.535936,2007-06,2007Q2,2007Q2
2008,14772.666854,5.726877,215.210798,2.052530,3.665771,99.671502,98.592055,63.449407,32.694862,2008-06,2008Q2,2008Q2
